## Design a recursion function 

### Interpret Recursion

Consider we want to solve a problem $F(X)$, we can decompose it into several small problems: $x_0 \in X, x_1 \in X, ....$

Base case: 
- **The solution of the minimum scale problem is the base case** of the recursive function, 
- through which the <u>pop operation</u> of the stack is realized until the stack is empty, and the problem is solved.
- In base case, the problem can be solved directly, and base case is usually taken as the jump out condition of recursion.

Recursion relationship: 
- **The relationship between the result of the problem and results of its sub-problems**
- Writing out the recursive relationship is equivalent to implementing the <u>push operation</u> of the stack. 

> **Recursion is a programming technique, not an algorithm**. *Divide and conquer, dynamic programming, and backtracking* are algorithms. The relationship between the two is that the most commonly used implementation of these three algorithms is recursion.<br/>
In both the divide and conquer algorithm and the dynamic programming algorithm, it is necessary to disassemble the large problem into sub-problems.

### **wrapper funtion**


A wrapper function is a function that is directly called but does not recurse itself, instead calling a separate *auxiliary function* which actually does the recursion.

Wrapper functions can be used to <u>validate parameters (so the recursive function can skip these)</u>, perform initialization (allocate memory, initialize variables), particularly for auxiliary variables such as "level of recursion" or <u>partial computations for memoization</u>, and handle exceptions and errors.

> Wrapper functions can reduce the number of parameters in the recursive call, which reduces the stack usage, as all parameters are pushed on the stack for each call, and if there are many parameters and many layers of recursion, there could be a stack overflow. Any parameters that do no change for the recursive call can be left in an enclosing lexical scope or passed altogether as a reference to a structure.

```c
\\ Ordinary
int fac1(int n) {
   if (n <= 0)
      return 1;
   else
      return fac1(n-1)*n;
}
```

```c
\\ Short-circuit recursion 
static int fac2(int n) {
   // assert(n >= 2);
   if (n == 2)
      return 2;
   else
      return fac2(n-1)*n;
}
int fac2wrapper(int n) {
   if (n <= 1)
      return 1;
   else
      return fac2(n);
}
```

**Short-circuiting the base case**:
Short-circuiting the base case, also known as **arm's-length recursion**, consists of checking the base case before making a recursive call – *i.e.*, checking if the next call will be the base case, instead of calling and then checking for the base case.

For example, in the factorial function, <u>properly the base case is 0! = 1, while immediately returning 1 for 1! is a short circuit</u>, and may miss 0; this can be mitigated by a wrapper function. 

- Short-circuiting is particularly done for efficiency reasons, to avoid the overhead of a function call that immediately returns.
- Short-circuiting is primarily a concern when many base cases are encountered, such as `Null` pointers in a tree, which can be linear in the number of function calls, hence significant savings for $O(n)$ algorithms; this is illustrated below for a depth-first search. 
- Short-circuiting on a tree corresponds to considering <u>a leaf (non-empty node with no children) as the base case</u>, rather than considering an empty node as the base case. If there is only a single base case, such as in computing the factorial, short-circuiting provides only O(1) savings. 


#### DFS

The standard recursive algorithm for a DFS is:

- base case: If current node is Null, return false
- recursive step: otherwise, check value of current node, return true if match, otherwise recurse on children

In short-circuiting, this is instead:

- check value of current node, return true if match,
- otherwise, on children, if not Null, then recurse.


In the case of a perfect binary tree of height $h$, there are $2^{h+1}−1$ nodes and $2^{h+1}$ `Null` pointers as children ($2$ for each of the $2^h$ leaves), so short-circuiting cuts the number of function calls in half in the worst case. 

```c
bool tree_contains(struct node *tree_node, int i) {
    if (tree_node == NULL)
        return false;  // base case
    else if (tree_node->data == i)
        return true;
    else
        return tree_contains(tree_node->left, i) ||
               tree_contains(tree_node->right, i);
}

```

```c
// Wrapper function to handle empty tree
bool tree_contains(struct node *tree_node, int i) {
    if (tree_node == NULL)
        return false;  // empty tree
    else
        return tree_contains_do(tree_node, i);  // call auxiliary function
}

// Assumes tree_node != NULL
bool tree_contains_do(struct node *tree_node, int i) {
    if (tree_node->data == i)
        return true;  // found
    else  // recurse
        return (tree_node->left  && tree_contains_do(tree_node->left,  i)) ||
               (tree_node->right && tree_contains_do(tree_node->right, i));
}
```

### **How to write a recursive function**

1. Find the basic case;
2. Find the recurrence relationship.

There are 2 paradigms we can follow: **from bottom to up, from up to bottom**.

#### From bottom to up

**Main idea**: first, call the recursive function itself; then, perform calculations based on the return value.

**Example**:
```java
/** 
 * Simulation process：
 * 5 + sum(4)
 * 5 + (4 + sum(3)
 * 5 + 4 + (3 + sum(2))
 * 5 + 4 + 3 + (2 + sum(1))
 * ------------------> jump out until sum(1) = 1
 * 5 + 4 + 3 + (2 + 1)
 * 5 + 4 + (3 + 3)
 * 5 + (4 + 6)
 * (5 + 10)
 * 15
 * from up to bottom: calculated as 1 + 2 + 3 + 4 + 5 
 */
public int sum(int n) {
    if (n < 2) return n;       // ① base case
    int childSum = sum(n - 1); // ② find recursive relationship
    return n + childSum;       // ③ calculate based on the return result
}
```

**Bottom-up paradigm**:
1. Find the base case and return the result of the base case when jumping out;
2. **Modify the parameters** of the recursive function;
3. Call the recursive function and get **intermediate variables**;
4. Use the result of the recursive function to calculate the final result with the current parameters;
5. Return the final result.

```python
def func(params):
    if (base case): return res of base case
    current params = modify(params)
    inter var = func(current params)
    final res = calculate(inter var, current params)
    return final res
```

#### From up to bottom

**Main idea**: calculate the intermediate variables based on current parameters, and then modify parameters, **pass these intermediate variables to the recursive function with modified parameters**, at last, **return the recursive function itself**.  

**Example**:
```java

/**
 * 模拟程序执行过程：
 * sum(5, 0)
 * sum(4, 5)
 * sum(3, 9)
 * sum(2, 12)
 * sum(1, 14)
 * 15
 * from up to bottom: calculated as  5 + 4 + 3 + 2 + 1 
 */
public int sum2(int n, int sum) {
    if (n < 2) return 1 + sum;
    sum += n;
    return sum2(n - 1, sum);
}
```

**Up-bottom paradigm**:
1. find the base case, and return the result of the base case (`1`) and calculation results of the intermediate variables (`sum`)
2. recalculate new intermediate variables based on current parameters.
3. Modify the parameters of the recursive function;
4. **Return the recursive function, with new intermediate variables and modified parameters as arguments.**

```python
def func(inter var, params):
    if (base case): return res of base case + inter var
    new inter var = calculate(inter var, params)
    new params = modify(params)
    return func(new inter var, new params)
```


#### Difference between Botton-up and Up-bottom

![](https://pic.leetcode-cn.com/367f77a847ab554d6c00a396b91d79a0c1effcbbc89d17d773e91a34345304e5-digui.jpg)

## Case Study

---
###  [Interview 08.05 Recursive multiplication](https://leetcode.cn/problems/recursive-mulitply-lcci/description/)

**Description**: Write a recursive function to multiply two positive integers without using the * operator. Plus, minus, shifts are fine, but be stingy. 

**Example**:
```
Input  : A = 1, B = 10 
  Output  : 10 
```


#### Plus


**Analysis**:
- Base case:
   when `A = 0` (or `B = 0`), return `B + sum` (or `A + sum`);
- Recursive relationship:
   `sum += B`, if `A` is taken as the counter; otherwise, `sum += A`

> **NB**: solve this problem by adding one by on, which may exceed the limit of the maximum recursion number, such as `A = 343525325` and `B = 1`.

**Bottom-up Paradigm**

In [1]:
def multiply(A: int, B: int) -> int:
    
    def multi_helper(A: int, B: int) -> int:
        if A == 1: return B
        childSum = B + multi_helper(A - 1, B)
        return childSum
    
    if A == 0 or B == 0: return 0
    if A > B: A, B = B, A
    return multi_helper(A, B)

A, B = 3, 4
multiply(A, B)

12

**Up-bottom Paradigm**

In [2]:
def multiply(A: int, B: int) -> int:
    
    def multi_helper(A: int, B: int, sum: int) -> int:
        if A == 1: return B + sum
        sum += B
        return multi_helper(A - 1, B, sum)

    if A == 0 or B == 0: return 0    
    if A > B: A, B = B, A
    return multi_helper(A, B, 0)

A, B = 3, 4
multiply(A, B)

12

#### Bitwise shift

**Analysis**:
- Base case: if `A = 1`, return `B`
- Recursive relationship: consider `A >> 1` when `A > 1`, there are 2 possible situation:
  - if `A` is an odd, , `A * B` = `(A >> 1) * (B << 1) + B`
  - if `A` is an even, `A * B` = `(A >> 1) * (B << 1)`

In [3]:
def multiply(A: int, B: int) -> int:
    if A == 0 or B == 0: return 0
    def multi_helper(A: int, B: int, sum: int) -> int:
        if A == 1: return sum + B
        if A % 2: sum += B
        A >>= 1
        B <<= 1
        return multi_helper(A, B, sum)
    if A > B: A, B = B, A
    return multi_helper(A, B, 0)

A, B = 14, 73807517
multiply(A, B)

1033305238

---
### [Offer-II 10. Frog Jumping](https://leetcode.cn/problems/qing-wa-tiao-tai-jie-wen-ti-lcof/)

**Description**: A frog can jump up 1 step or 2 steps at a time. Ask the frog to jump on a `n`-levels stair, then, how many combinations of jumping are there in total? (
`0 <= n <= 100`)

> The answer needs to be modulo with `1e9+7` (1000000007), *e.g.*, if the initial calculation result is: 1000000008, please return 1.

**Example**:
```
Input:  n = 7 
  Output:  21 
```

```
Input:  n = 2 
  Output:  2
```

```
Input:  n = 0 
  Output:  1
```

**Analysis**:
- Base case:
  if there's only 1 step left or no step, then the frog has only one choice.
- Recursive relationship:
  - Strategies counts 1: after jumping 2 steps, `nums = func(n - 2)`;
  - Strategies counts 2: after jumping 1 step, `nums = func(n - 1)`;
  - Total counts: counts 1 + counts 2.

> **NB**: in order to avoid calculate the counts of the same status, it is necessary to store the calculated status into a set / a dictionary. <br/>
> For example, Strategy 1: `{step1 = 1, step2 = 1, step3 = 1}` and Strategy 2: `{step1 = 2, step2 = 1}`, have the same status, which is the remaining steps being `n - 3`.

**Bottom-up Paradigm**:

In [4]:
def numWays(n: int) -> int:
    
    def nums_helper(n: int) -> int:
        if n == 0 or n == 1: return 1
        count = status_dict.get(n)
        if count is not None: return count 
        nums1 = nums_helper(n - 1)
        nums2 = nums_helper(n - 2)
        count = nums2 + nums1
        status_dict[n] = count
        return count
    
    status_dict = {}
    return nums_helper(n) % int(1e9+7)
    
n = 44
numWays(n)

134903163

**Up-bottom Paradigm**:

In bottom-up paradigm (from $n \rightarrow 1$), the relation can be expressed as $f(n) = f(n-1) + f(n-2)$; On the contrary, in up-bottom paradigm (from $1 \rightarrow n$), the relation can be expressed as $f(i + 1) = f(i) + f(i-1)$.

Assume $f(i-1) = a, f(i-2) = b$, we can have:
$$
f(i) = f(i -1) + f(i - 2) = a + b \\
f(i + 1) = f(i) + f(i - 1) = (a + b) + a
$$

In [5]:
def numWays(n: int) -> int:
    
    def nums_helper(n: int, i: int, a: int, b: int) -> int:
        if i >= n: return a + b
        return nums_helper(n, i + 1, a + b, a)
    
    if n <= 1: return 1
    return nums_helper(n, 2, 1, 1) % int(1e9 + 7)

n = 44
numWays(n)

134903163

---
### [Offer 25. Merge 2 sorted linked lists](https://leetcode.cn/problems/he-bing-liang-ge-pai-xu-de-lian-biao-lcof/)

**Description**: Enter two ascending linked lists, merge the two linked lists and keep the nodes in the new linked list still in ascending order. 

**Examples**: 
```
Input:  1->2->4, 1->3->4 
  Output:  1->1->2->3->4->4 
```

**Linked list initialization**:

In [45]:
class ListNode:
    def __init__(self, val: int):
        self.val = val
        self.next = None

    def __repr__(self):
        current_node = self
        string = ''
        while current_node is not None:
            if current_node.next is not None:
                string += f"{current_node.val} -> "
                current_node = current_node.next
            else:
                string += f"{current_node.val}"
                return string
            
    def clone(self):
        other = ListNode(self.val) 
        stack = [self]
        stack2 = [other]
        while stack or stack2:
            node = stack.pop()
            node2 = stack2.pop()
            if node.next is not None:
                node2.next = ListNode(node.next.val)
                stack.append(node.next)
                stack2.append(node2.next)
        return other


**Analysis**:
- Base case:
  - `l1` or `l2` is empty, return another linked list.
  - `l1` has only one element, traverse `l2` to find the proper position.
- Recursive relationship:
  - assume `l1.headNode < l2.headNode`, `res[-1].next = l2.headNode`
  - `l1.headNode = l1.headNode.next`
  - otherwise, put `l2.headNode` to the merged linked list.

**Bottom-up Paradigm**:

In [44]:
def mergeTwoLists(l1: ListNode, l2: ListNode) -> ListNode:
    # base case
    if not l1 and not l2: return None
    if not l1: return l2 
    if not l2: return l1
    # obtain the intermediate variable by calling function recursively  
    if l1.val < l2.val:
        res = mergeTwoLists(l1.next, l2)
        l1.next = res
        return l1
    else:
        res = mergeTwoLists(l1, l2.next)
        l2.next = res
        return l2

In [42]:
node11, node12, node13 = ListNode(1), ListNode(2), ListNode(6)
node11.next = node12
node12.next = node13

node21, node22, node23 = ListNode(1), ListNode(3), ListNode(5)
node21.next = node22
node22.next = node23

print(node11), print(node21)

1 -> 2 -> 6
1 -> 3 -> 5


(None, None)

In [51]:
l1, l2 = node11.clone(), node21.clone()
res = mergeTwoLists(l1, l2)
print(res)

1 -> 1 -> 2 -> 3 -> 5 -> 6
